In [1]:
# Import dependcies
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Create global seed
yogi = 8

In [2]:
# Read in data 
file_path = '../Data/Pitchers/lamet.csv'
lamet_df = pd.read_csv(file_path)

lamet_df.dropna(inplace = True)

lamet_df.head(10)

,Unnamed: 0,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,...,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp
0,13042,SL,84.8,-1.44,6.25,"Lamet, Dinelson",14.0,R,R,R,...,84.9,2551.0,6.3,54.16,27,1,Slider,63.0,0.000,0.040
1,13043,SL,85.4,-1.34,6.23,"Lamet, Dinelson",9.0,R,R,R,...,85.5,2341.0,6.2,54.30,26,6,Slider,127.0,-0.038,-0.611
2,13044,FF,95.3,-1.18,6.21,"Lamet, Dinelson",12.0,R,R,R,...,95.0,2413.0,6.0,54.47,26,5,4-Seam Fastball,203.0,0.000,0.131
3,13045,SL,84.3,-1.30,6.32,"Lamet, Dinelson",14.0,R,R,R,...,84.0,2648.0,6.2,54.25,26,4,Slider,63.0,0.000,0.043
4,13046,SL,81.1,-1.38,6.36,"Lamet, Dinelson",6.0,R,R,R,...,80.7,2461.0,6.2,54.28,26,3,Slider,49.0,0.000,-0.098
5,13047,FF,95.7,-1.14,6.22,"Lamet, Dinelson",14.0,R,R,R,...,95.5,2417.0,6.2,54.25,26,2,4-Seam Fastball,209.0,0.000,0.074
6,13048,SL,84.6,-1.39,6.25,"Lamet, Dinelson",6.0,R,R,R,...,83.7,2512.0,6.1,54.43,26,1,Slider,45.0,0.000,-0.078
7,13049,SL,87.0,-1.31,6.19,"Lamet, Dinelson",14.0,R,L,R,...,86.2,2422.0,6.0,54.50,25,7,Slider,178.0,-0.053,0.363
8,13050,SL,87.0,-1.34,6.14,"Lamet, Dinelson",11.0,R,L,R,...,87.0,2342.0,6.1,54.42,25,6,Slider,165.0,0.000,0.000
9,13051,SL,85.9,-1.43,6.23,"Lamet, Dinelson",1.0,R,L,R,...,85.7,2366.0,6.1,54.43,25,5,Slider,142.0,0.000,-0.127


In [3]:
# Drop Unnamed columns
lamet_df.drop(['Unnamed: 0', 'pitch_name'], axis = 1, inplace = True)

In [4]:
# Generate categorical variable list
pitch_cat = lamet_df.dtypes[lamet_df.dtypes == 'object'].index.tolist()
pitch_cat.remove('player_name')
pitch_cat

['pitch_type', 'game_type', 'stand', 'p_throws', 'type']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(lamet_df[pitch_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(pitch_cat)
encode_df.head()

,pitch_type_CH,pitch_type_CU,pitch_type_FF,pitch_type_FT,pitch_type_SI,pitch_type_SL,game_type_R,stand_L,stand_R,p_throws_R,type_B,type_S,type_X
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
lamet_df = lamet_df.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(pitch_cat, 1)

lamet_df.head()

,release_speed,release_pos_x,release_pos_z,player_name,zone,balls,strikes,pfx_x,pfx_z,plate_x,...,pitch_type_FT,pitch_type_SI,pitch_type_SL,game_type_R,stand_L,stand_R,p_throws_R,type_B,type_S,type_X
0,84.8,-1.44,6.25,"Lamet, Dinelson",14.0,0,0,0.00,0.21,1.57,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,85.4,-1.34,6.23,"Lamet, Dinelson",9.0,3,2,-0.15,0.33,0.57,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,95.3,-1.18,6.21,"Lamet, Dinelson",12.0,2,2,-0.72,1.50,1.34,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,84.3,-1.30,6.32,"Lamet, Dinelson",14.0,1,2,0.13,0.23,0.78,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,81.1,-1.38,6.36,"Lamet, Dinelson",6.0,1,1,0.89,0.12,0.58,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [7]:
# Drop superfluous columns 
lamet_df.drop(columns = ['stand_L'], axis = 1, inplace = True)
lamet_df.head()


,release_speed,release_pos_x,release_pos_z,player_name,zone,balls,strikes,pfx_x,pfx_z,plate_x,...,pitch_type_FF,pitch_type_FT,pitch_type_SI,pitch_type_SL,game_type_R,stand_R,p_throws_R,type_B,type_S,type_X
0,84.8,-1.44,6.25,"Lamet, Dinelson",14.0,0,0,0.00,0.21,1.57,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,85.4,-1.34,6.23,"Lamet, Dinelson",9.0,3,2,-0.15,0.33,0.57,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
2,95.3,-1.18,6.21,"Lamet, Dinelson",12.0,2,2,-0.72,1.50,1.34,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
3,84.3,-1.30,6.32,"Lamet, Dinelson",14.0,1,2,0.13,0.23,0.78,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,81.1,-1.38,6.36,"Lamet, Dinelson",6.0,1,1,0.89,0.12,0.58,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0


### Compiling, Training, and Testing Data

In [8]:
# Define features set
X = lamet_df.copy()
X.drop(columns = ['player_name', 'zone', 'balls', 'strikes', 'delta_run_exp', 'delta_home_win_exp'], axis = 1, inplace = True)
X.head()

,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,...,pitch_type_FF,pitch_type_FT,pitch_type_SI,pitch_type_SL,game_type_R,stand_R,p_throws_R,type_B,type_S,type_X
0,84.8,-1.44,6.25,0.00,0.21,1.57,1.35,4,7.030834,-123.354348,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,85.4,-1.34,6.23,-0.15,0.33,0.57,1.57,4,4.791514,-124.301899,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
2,95.3,-1.18,6.21,-0.72,1.50,1.34,2.71,4,8.201043,-138.553164,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
3,84.3,-1.30,6.32,0.13,0.23,0.78,-0.04,4,4.565520,-122.503162,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,81.1,-1.38,6.36,0.89,0.12,0.58,2.67,4,2.670614,-118.028404,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0


In [9]:
# Define target vector
y = lamet_df['zone'].values
y[:5]

array([14.,  9., 12., 14.,  6.])

In [10]:
# Split the data into training and testing sets - stratify by pitcher
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = yogi)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit Standard Scaler 
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Find Best Learning Rate

In [12]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
best_rate = 0
best_acc = 0

for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(
        n_estimators = 20,
        learning_rate = learning_rate,
        max_features = 7,
        max_depth = 4,
        random_state = yogi)
    
    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print(f'Learning Rate: {learning_rate}')

    # Score the model
    print('Accuracy Score (training): {0:.3f}'.format(
        classifier.score(
            X_train_scaled,
            y_train
        )
    ))
    print('Accuracy Score (validation): {0:.3f}'.format(
        classifier.score(
            X_test_scaled,
            y_test
        )
    ))
    print()

    if classifier.score(X_test_scaled, y_test) > best_acc:
        best_acc = classifier.score(X_test_scaled, y_test)
        best_rate = learning_rate

Learning Rate: 0.05
Accuracy Score (training): 0.954
Accuracy Score (validation): 0.819

Learning Rate: 0.1
Accuracy Score (training): 0.981
Accuracy Score (validation): 0.848

Learning Rate: 0.25
Accuracy Score (training): 1.000
Accuracy Score (validation): 0.864

Learning Rate: 0.5
Accuracy Score (training): 1.000
Accuracy Score (validation): 0.841

Learning Rate: 0.75
Accuracy Score (training): 0.749
Accuracy Score (validation): 0.556

Learning Rate: 1
Accuracy Score (training): 0.066
Accuracy Score (validation): 0.054



### Create model with best learning rate

In [13]:
# Choose a learning rate and create classifiers
classifier = GradientBoostingClassifier(
    n_estimators = 20,
    learning_rate = best_rate,
    max_features = 7,
    max_depth = 3,
    random_state = yogi
)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make predictions
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).head(20)

,Prediction,Actual
0,1.0,4.0
1,6.0,6.0
2,8.0,8.0
3,14.0,14.0
4,14.0,14.0
5,5.0,5.0
6,9.0,9.0
7,5.0,5.0
8,11.0,11.0
9,11.0,1.0


### Evaluate Model

In [14]:
# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.8512544802867383


In [15]:
# Generate classification report
print('Classification Report')
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

         1.0       0.71      0.60      0.65        20
         2.0       0.77      0.59      0.67        29
         3.0       0.80      0.86      0.83        28
         4.0       0.62      0.84      0.71        25
         5.0       0.75      0.87      0.80        47
         6.0       0.84      0.90      0.87        48
         7.0       0.50      0.33      0.40        18
         8.0       0.81      0.87      0.84        45
         9.0       0.86      0.89      0.87        35
        11.0       0.91      0.90      0.91        59
        12.0       0.88      0.80      0.84        35
        13.0       0.94      0.75      0.83        20
        14.0       0.99      0.97      0.98       149

    accuracy                           0.85       558
   macro avg       0.80      0.78      0.78       558
weighted avg       0.85      0.85      0.85       558

